# Using the qiskit interface to perform mGST

In this notebook we will generate random sequences of gates, create qiskit circuits from them, and simulate them on a qiskit simulator.
Then we estimate the gates.

In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
from mGST import qiskit_interface
from qiskit.circuit.library import RXGate, RYGate, RZZGate, IGate
from mGST.qiskit_interface import job_counts_to_mgst_format, get_gate_estimation
from qiskit_aer import Aer

backend = Aer.get_backend("qasm_simulator")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Example usage with RX, RY, and RZZ gates
rx_pi_2 = RXGate(np.pi / 2)
rx_pi = RXGate(np.pi)
ry_pi_2 = RYGate(np.pi / 2)
ry_pi = RYGate(np.pi)
rzz_pi_2 = RZZGate(np.pi / 2)
idle_gate = IGate()

gate_set = [idle_gate, rx_pi, ry_pi, rx_pi_2, ry_pi_2,]
K_true = qiskit_interface.qiskit_gate_to_operator(gate_set)

In [9]:
sequence_number = 100  # Number of sequences
shots = 1000  # Number of samples per sequences
sequence_length = 7 # The depth of randomly generated circuits for GST
n_qubits = 1 # The total number of qubits on the backend
active_qubits = [0] # The qubits on which the gate set is defined
n_povm = 2**(len(active_qubits)) # The number of measurement outcomes

In [10]:
# Running the circuits and formating the result
gate_sequences = qiskit_interface.get_gate_sequence(sequence_number, sequence_length, len(gate_set))
qiskit_circuits = qiskit_interface.get_qiskit_circuits(gate_sequences, gate_set, n_qubits, active_qubits)
job_counts = backend.run(qiskit_circuits, shots=shots).result().get_counts()
sequence_results = job_counts_to_mgst_format(active_qubits, n_povm, job_counts)

In [11]:
# Running GST
X_opt, E_opt, rho_opt = get_gate_estimation(gate_set, gate_sequences, sequence_results, shots, rK = 4)

Starting mGST optimization...
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]
Batch optimization successful, improving estimate over full data....
 12%|█████████▉                                                                         | 6/50 [00:08<01:00,  1.37s/it]
	 Convergence criterion satisfied
	 Final objective 1.16e-4 with # of initializations: 1 in time 8.95s
Mean variation error: 0.03610840431580817
Optimizing gauge...
Compressive GST routine complete
First results:
           F_avg  Min. Spectral distances
Gate 0  0.997002                 0.005827
Gate 1  0.999585                 0.000689
Gate 2  0.999469                 0.000789
Gate 3  0.999972                 0.001031
Gate 4  0.999972                 0.003583
                                
Final cost function     0.000116
Mean TVD estimate-data  0.008026
Mean TVD target-data    0.007950
SPAM error:             0.007711
